In [1]:
import numpy as np
import json
import random
import sklearn.cluster as sk_cluster
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from scipy.spatial import distance
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score

In [2]:
global ingredients_sort # list with sorted pairs of kay,values in descending order according to value
global subset # list of recipes dicts, with missing ingredient recipes
global unique_ingredients # ingredients list
global original_recipes # 1000 recipes in original form, these recipes miss an ingredient in M
global jaccard_recipes # jaccard score matrix for recipes

## Create the data

In [3]:
f = open("train.json")
data = json.load(f)
len(data)

39774

Keep only italian and non empty ingresients lists

In [4]:
subset = [i for i in data if i["cuisine"]=="italian" and
          not i["ingredients"] is None]

Make a list with all the ingredients in our dataset (only one entry for every ingredient)

In [5]:
ingredients_list = [np.array(i["ingredients"]) for i in subset]
unique_ingredients = np.concatenate(ingredients_list)
unique_ingredients = np.sort(np.unique(unique_ingredients))

Pick 1000 random indexes to remove a single ingredient from each recipe that points

In [6]:
random_indexes = random.sample(range(0, len(subset)-1), 1000)

In [7]:
original_recipes = []
for i in random_indexes:
    # Copy the recipe and sort the ingredients list
    temp = subset[i].copy()
    temp["index"] = i
    temp["ingredients"] = np.sort(temp["ingredients"])
    # Save the original recipe and its index
    original_recipes.append(temp)
    # Shuffle the ingredients and remove the last and sort them again
    random.shuffle(subset[i]["ingredients"])
    subset[i]["ingredients"] = np.sort(subset[i]["ingredients"][:-1])
    
    

print(original_recipes[0]["ingredients"])
print(subset[original_recipes[0]["index"]]["ingredients"])

['baby spinach' 'capers' 'champagne vinegar' 'ciabatta'
 'cream cheese, soften' 'dijon mustard' 'extra-virgin olive oil'
 'fresh lemon juice' 'grated lemon zest' 'ground black pepper'
 'minced garlic' 'purple onion' 'salt' 'sliced green onions'
 'smoked salmon' "soft goat's cheese"]
['baby spinach' 'capers' 'champagne vinegar' 'ciabatta'
 'cream cheese, soften' 'dijon mustard' 'extra-virgin olive oil'
 'fresh lemon juice' 'grated lemon zest' 'ground black pepper'
 'minced garlic' 'purple onion' 'sliced green onions' 'smoked salmon'
 "soft goat's cheese"]


Create italian_recipes * unique_ingredients matrix (This is M matrix that is created from the subset that we removed the ingredients from 1000 recipes)

In [8]:
M = np.zeros((len(subset), len(unique_ingredients)))

for x, r in enumerate(subset):
    for i in r["ingredients"]:
        y = np.where(unique_ingredients == i)
        M[x, y] = 1

In [9]:
len(random_indexes)

1000

In [10]:
R = np.zeros((len(random_indexes), len(unique_ingredients)))
for k, x in enumerate(random_indexes):
    for i in subset[x]["ingredients"]:
        y = np.where(unique_ingredients == i)
        R[k, y] = 1


In [11]:
R.shape

(1000, 2929)

Create a dictionary with the ingredients frequencies.

In [12]:
ingredients_dict = {}

for r in subset:
    for i in r["ingredients"]:
        if i in ingredients_dict.keys():
            ingredients_dict[i] += 1
        else:
            ingredients_dict[i] = 1


Create a sorted by value list to store  (key, frequency) in descending order.

In [13]:
ingredients_sort = sorted(ingredients_dict.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)

Jaccard Similarity Matrix (Recipes)

In [24]:
jaccard_recipes = metrics.pairwise.pairwise_distances(R,M,metric = 'jaccard')

/home/jwick/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


## Score Formulas

### General purpose functions

In [17]:
def score(r, i , formula):
    
    if formula == "mp":
        return mp(r,i)
        # call most popular
    elif formula == "ucf":
        return 1
        # call micf
    elif formula == "icf":
        return 2
        # call icf
    else:
        print("Wrong formula input.")

In [18]:
def get_missing_ingredient(r):
    for k, i in enumerate(original_recipes[r]["ingredients"]):
        if i == subset[original_recipes[r]["index"]]["ingredients"][k]:
            return i

In [43]:
def get_top_N(in_array, N):
    args = np.argsort(in_array)
    args = list(reversed(args[-N:]))
    return args

In [44]:
get_top_N(np.array([1,3,5,6,3,2,1]), 5)

[3, 2, 4, 1, 5]

In [ ]:
def get_B_N()

### Most Popular Ingredient (MP)

In [19]:
def mp(r, i):
    for p in ingredients_sort:
        if not p[0] in subset[original_recipes[r]["index"]]["ingredients"]:
            print("Expecting: ", get_missing_ingredient(r))
            print("Returned:  ", p[0])
            return p

In [20]:
mp(100, "salt")

Expecting:  all purpose unbleached flour
Returned:   garlic cloves


('garlic cloves', 1601)

### User-based Collaborative filtering (UCF)

In [21]:
def ucf(r, i):
    for 

SyntaxError: unexpected EOF while parsing (<ipython-input-21-2a435afafe6f>, line 2)